In [ ]:
!pip install numpy torch
!pip install git+https://github.com/brain-score/brain-score.git
!pip install git+https://github.com/openai/CLIP.git
!pip install git+https://github.com/brain-score/model-tools.git

In [4]:
from model_tools.check_submission import check_models
import numpy as np
import torch
from torch import nn
import functools
from model_tools.activations.pytorch import PytorchWrapper
from model_tools.brain_transformation import ModelCommitment
from model_tools.activations.pytorch import load_preprocess_images
from brainscore import score_model

import clip

model_definitions = {
    'RN50': {'layers': ['layer2.3.bn3', 'layer3.5.bn3', 'layer4.2.bn3', ]},
    'RN101': {'layers': ['layer2.3.bn3', 'layer3.22.bn3', 'layer4.2.bn3', ]},
    'RN50x4': {'layers': ['layer2.5.bn3', 'layer3.9.bn3', 'layer4.5.bn3', ]},
    'RN50x16': {'layers': ['layer2.7.bn3', 'layer3.17.bn3', 'layer4.7.bn3', ]},
    'ViT-B/32': {'layers': [*[f'transformer.resblocks.{i}' for i in range(12)], 'ln_post']},
    'ViT-B/16': {'layers': [*[f'transformer.resblocks.{i}' for i in range(12)], 'ln_post']},
}

available_models = clip.available_models()
for model_name in available_models:
    if model_name not in model_definitions:
        raise Warning(f"New clip model available which is not defined: {model_name}")


for model_name, model_definition in model_definitions.items():
    if model_name not in available_models:
        raise Warning(f"Clip model {model_name} is not available, skipping it")
        continue

    clip_model, preprocess = clip.load(model_name, device='cpu')

    resize = preprocess.transforms[0]
    normalize = preprocess.transforms[4]
    preprocessing = functools.partial(load_preprocess_images,
                                      image_size=resize.size,
                                      normalize_mean=normalize.mean,
                                      normalize_std=normalize.std)

    model_definition['activations'] = PytorchWrapper(identifier=model_name, model=clip_model.visual, preprocessing=preprocessing)
    model_definition['model'] = ModelCommitment(identifier=model_name,
                                                activations_model=model_definition['activations'],
                                                layers=model_definition['layers'],)
                                                #behavioral_readout_layer=model_definition['layers'][-1])

In [5]:
from brainscore.benchmarks import public_benchmark_pool

public_benchmark_pool

Loading lookup from entrypoints
Loading lookup from /usr/local/lib/python3.7/dist-packages/brainscore/lookup.csv


brainio.dicarlo/assy_dicarlo_MajajHong2015_public.nc: 100%|██████████| 165M/165M [00:01<00:00, 124MB/s]
brainio.dicarlo/image_dicarlo_hvm-public.csv: 100%|██████████| 1.33M/1.33M [00:00<00:00, 10.6MB/s]
brainio.dicarlo/image_dicarlo_hvm-public.zip: 100%|██████████| 132M/132M [00:01<00:00, 106MB/s]
brainio.dicarlo/image_dicarlo_objectome_public.csv: 100%|██████████| 270k/270k [00:00<00:00, 3.06MB/s]
brainio.dicarlo/image_dicarlo_objectome_public.zip: 100%|██████████| 79.6M/79.6M [00:00<00:00, 114MB/s]
brainio-contrib/image_dietterich_Hendrycks2019_blur.csv: 100%|██████████| 123M/123M [00:01<00:00, 105MB/s]
brainio-contrib/image_dietterich_Hendrycks2019_blur.zip: 100%|██████████| 7.49G/7.49G [00:57<00:00, 131MB/s]
brainio-contrib/image_dietterich_Hendrycks2019_digital.csv: 100%|██████████| 128M/128M [00:01<00:00, 106MB/s]
brainio-contrib/image_dietterich_Hendrycks2019_digital.zip: 100%|██████████| 8.15G/8.15G [01:02<00:00, 130MB/s]
brainio-contrib/image_dietterich_Hendrycks2019_noise.csv

{'dicarlo.MajajHong2015public.IT-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x7fcbb067e5d0>,
 'dicarlo.MajajHong2015public.V4-pls': <brainscore.benchmarks._neural_common.NeuralBenchmark at 0x7fcbb067e190>,
 'dicarlo.Rajalingham2018public-i2n': <brainscore.benchmarks.public_benchmarks.RajalinghamMatchtosamplePublicBenchmark at 0x7fcbb067e9d0>,
 'dietterich.Hendrycks2019-blur-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x7fcbb0d82c90>,
 'dietterich.Hendrycks2019-digital-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x7fcbb0651f90>,
 'dietterich.Hendrycks2019-noise-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x7fcbb0d7a6d0>,
 'dietterich.Hendrycks2019-weather-top1': <brainscore.benchmarks.imagenet_c.Imagenet_C_Category at 0x7fcbb0651150>,
 'fei-fei.Deng2009-top1': <brainscore.benchmarks.imagenet.Imagenet2012 at 0x7fcbb0d7a710>,
 'movshon.FreemanZiemba2013public.V1-pls': <brainscore.benchmarks._neural_common.Neur

In [ ]:
from brainscore.benchmarks import public_benchmark_pool

benchmark = public_benchmark_pool['dicarlo.MajajHong2015public.IT-pls']

score = benchmark(model_definitions['RN50']['model'])

Loading lookup from entrypoints
Loading lookup from /usr/local/lib/python3.7/dist-packages/brainscore/lookup.csv


activations:   0%|          | 0/3200 [00:00<?, ?it/s]

activations:   0%|          | 0/1024 [00:00<?, ?it/s]

activations:   6%|▋         | 64/1024 [00:15<03:51,  4.15it/s]

activations:  12%|█▎        | 128/1024 [00:30<03:30,  4.25it/s]

activations:  19%|█▉        | 192/1024 [00:45<03:15,  4.25it/s]

activations:  25%|██▌       | 256/1024 [01:00<02:59,  4.28it/s]

activations:  31%|███▏      | 320/1024 [01:15<02:44,  4.28it/s]

activations:  38%|███▊      | 384/1024 [01:29<02:28,  4.30it/s]

activations:  44%|████▍     | 448/1024 [01:44<02:13,  4.31it/s]

activations:  50%|█████     | 512/1024 [01:59<01:59,  4.29it/s]

activations:  56%|█████▋    | 576/1024 [02:14<01:44,  4.28it/s]

activations:  62%|██████▎   | 640/1024 [02:30<01:30,  4.24it/s]

activations:  69%|██████▉   | 704/1024 [02:45<01:15,  4.23it/s]

activations:  75%|███████▌  | 768/1024 [03:00<01:00,  4.22it/s]

activations:  81%|████████▏ | 832/1024 [03:17<00:47,  4.08it/s]

activations:  88%|████████▊ | 896/1024 [03:36<

In [1]:
score

NameError: ignored